In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("data/cleaned/hate_speech_dataset.tsv")
df["cleaned_post"] = df["cleaned_post"].astype(str)

print(df["class"].value_counts())


df["implicit_binary"] = (df["class"] == "implicit_hate").astype(int)

print("\nNew binary label distribution (implicit_binary):")
print(df["implicit_binary"].value_counts())

X_text = df["cleaned_post"].values
y = df["implicit_binary"].values.astype("int32")

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\nTrain label distribution:", np.bincount(y_train))
print("Test label distribution :", np.bincount(y_test))

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

tfidf_vectorizer = TfidfVectorizer(
    use_idf=True,
    max_features=20000,
    ngram_range=(1, 2)  # unigram + bigram
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

nb_clf = MultinomialNB(alpha=0.1)
nb_clf.fit(X_train_tfidf, y_train)

y_pred_nb = nb_clf.predict(X_test_tfidf)




class
not_hate         13270
implicit_hate     7094
explicit_hate     1086
Name: count, dtype: int64

New binary label distribution (implicit_binary):
implicit_binary
0    14356
1     7094
Name: count, dtype: int64

Train label distribution: [11485  5675]
Test label distribution : [2871 1419]


In [4]:
print("\n==========================")
print(" Naive Bayes (Implicit vs Others)")
print("==========================\n")

print("Classification Report (Naive Bayes):")
print(classification_report(y_test, y_pred_nb, digits=4))

print("Confusion Matrix (Naive Bayes):")
print(confusion_matrix(y_test, y_pred_nb))


 Naive Bayes (Implicit vs Others)

Classification Report (Naive Bayes):
              precision    recall  f1-score   support

           0     0.7328    0.8788    0.7992      2871
           1     0.5891    0.3517    0.4404      1419

    accuracy                         0.7044      4290
   macro avg     0.6610    0.6152    0.6198      4290
weighted avg     0.6853    0.7044    0.6805      4290

Confusion Matrix (Naive Bayes):
[[2523  348]
 [ 920  499]]


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM, Dense, Dropout
)
from tensorflow.keras.callbacks import EarlyStopping

print("\nTensorFlow version:", tf.__version__)

MAX_WORDS = 20000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN)

print("Train padded shape:", X_train_pad.shape)
print("Test padded shape :", X_test_pad.shape)

EMBED_DIM = 100
LSTM_UNITS = 128
DROPOUT_RATE = 0.5

inputs = Input(shape=(MAX_LEN,))
x = Embedding(input_dim=MAX_WORDS,
              output_dim=EMBED_DIM,
              input_length=MAX_LEN)(inputs)

x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=False))(x)
x = Dropout(DROPOUT_RATE)(x)

output = Dense(1, activation="sigmoid")(x)

bilstm_model = Model(inputs=inputs, outputs=output)
bilstm_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

bilstm_model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history = bilstm_model.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)




TensorFlow version: 2.20.0
Train padded shape: (17160, 100)
Test padded shape : (4290, 100)


/opt/anaconda3/envs/dsan6600/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 100)       │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,234,753 (8.52 MB)

 Trainable params: 2,234,753 (8.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 53s 207ms/step - accuracy: 0.6901 - loss: 0.5948 - val_accuracy: 0.7121 - val_loss: 0.5647
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 63s 260ms/step - accuracy: 0.7870 - loss: 0.4485 - val_accuracy: 0.6976 - val_loss: 0.5932
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 48s 199ms/step - accuracy: 0.8603 - loss: 0.3237 - val_accuracy: 0.6882 - val_loss: 0.7037


In [6]:
loss_rnn, acc_rnn = bilstm_model.evaluate(X_test_pad, y_test, verbose=0)
print("\n==========================")
print(" Bi-LSTM (Implicit vs Others)")
print("==========================\n")
print(f"Test Accuracy (Bi-LSTM): {acc_rnn:.4f}")
print(f"Test Loss (Bi-LSTM): {loss_rnn:.4f}")

y_prob_rnn = bilstm_model.predict(X_test_pad)
y_pred_rnn = (y_prob_rnn >= 0.5).astype("int32").ravel()

print("\nClassification Report (Bi-LSTM):")
print(classification_report(y_test, y_pred_rnn, digits=4))

print("Confusion Matrix (Bi-LSTM):")
print(confusion_matrix(y_test, y_pred_rnn))


 Bi-LSTM (Implicit vs Others)

Test Accuracy (Bi-LSTM): 0.7166
Test Loss (Bi-LSTM): 0.5590
135/135 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step

Classification Report (Bi-LSTM):
              precision    recall  f1-score   support

           0     0.7345    0.9028    0.8100      2871
           1     0.6334    0.3397    0.4422      1419

    accuracy                         0.7166      4290
   macro avg     0.6839    0.6212    0.6261      4290
weighted avg     0.7010    0.7166    0.6883      4290

Confusion Matrix (Bi-LSTM):
[[2592  279]
 [ 937  482]]


In [ ]:
import numpy as np
import torch
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import classification_report, confusion_matrix

MODEL_NAME = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

MAX_LEN_BERT = 128

train_encodings = tokenizer(
    list(X_train_text),
    truncation=True,
    padding=True,
    max_length=MAX_LEN_BERT
)

test_encodings = tokenizer(
    list(X_test_text),
    truncation=True,
    padding=True,
    max_length=MAX_LEN_BERT
)

y_train_np = np.array(y_train, dtype=np.int64)
y_test_np = np.array(y_test, dtype=np.int64)


class ImplicitHateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = ImplicitHateDataset(train_encodings, y_train_np)
test_dataset = ImplicitHateDataset(test_encodings, y_test_np)


# 二分类用 num_labels=2
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


batch_size = 16

training_args = TrainingArguments(
    output_dir="./bert_implicit_results",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,

    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",

    logging_steps=50,

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)



from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

pred_output = trainer.predict(test_dataset)
logits = pred_output.predictions
y_pred_bert = np.argmax(logits, axis=-1)

print("\n==========================")
print(" BERT (Implicit vs Others)")
print("==========================\n")

print("Classification Report (BERT):")
print(classification_report(y_test_np, y_pred_bert, digits=4))

print("Confusion Matrix (BERT):")
print(confusion_matrix(y_test_np, y_pred_bert))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/_g/92751mbs51g14bh9sfb8q4bw0000gn/T/ipykernel_9960/656854219.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/anaconda3/envs/dsan6600/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.550300,0.529626,0.734965,0.619088,0.516561,0.563196
2,0.444600,0.537955,0.735664,0.589118,0.663848,0.624254


/opt/anaconda3/envs/dsan6600/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/dsan6600/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
